## Домашнее здание №2 / Link Prediction

### Матушкин Александр, 399 группа

----
План выполнения домашнего задания:

1. Краткий обзор исходных данных
2. Генерация дополнительных данных
3. Создание матрицы признаков для ребер
4. Настройка модели и валидация качества модели
5. Отправка результатов в контест на Kaggle

----

В данном домашнем задании вам предстоит построить классификатор, который бы предсказывал наличие или отсутствия ребра в графе между двумя вершинами. Никакой дополнительной информации о вершинах, кроме ее соседей нет, поэтому вам придется создавать вектор признаков для каждой пары вершин на основе топологии графа.

Вершины графа - товары интернет магазина, наличие ребра символизируют о высокой частоте совместной покупки данных товаров вместе. Например если в нашем графе присутствую вершины молоко и хлеб и данные товары часто покупаются вместе, то между данными вершинами будет ребро. Если в нашем графе есть вершины молоко и кроссовки, то между данными вершинами ребра не будет. 

В данном датасете уровень относительной частоты по которому решается есть ребро между вершинами или его нет был определен за нас. Сам датасет был получен краулингом сайта Amazon.com в марте 2003 года, парсилась секция "Люди которые купили данный продукт, также преобретали это..."

Мотивация данного задания: Расширить список блока рекомендаций, за счет товаров которые с высокой вероятность могут оказаться в одной корзине покупателя.

----
Для создания модели и работы с данными мы будем использовать пакет GraphLab, структуры данных SFrame и SGraph идеально подходят для работы с графами. Распределенное хранение данных и применения функций для расчета метрик отдельных вершин сильно облегчают работу с графом.

Библиотека платная, но лицензия для академических целей получается в течении 5 минут, чтобы установить пакет следуйте шагам на сайте - https://turi.com/download/academic.html

----

Описание файлов:

1. the_graph.csv - файл содержащий ребра графа, две колонки: src,dst 
2. suspicions.csv - файл с ребрами, графа. Для данных ребер неизвестно присутствует ли оно в графе или нет.

Описание целевой метрики - в качестве целевой метрики будем использовать ROC AUC http://mlwiki.org/index.php/ROC_Analysis

----

In [1]:
import graphlab as gl
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

gl.canvas.set_target('ipynb')

### 1. Краткий обзор исходных данных - 10 Баллов

Загрузим данные, найдем ряд базовых статистик:

1. Количество вершин и ребер
2. Распределение степеней вершин графа (График log - log)
3. Плотность графа
4. Диаметр графа
5. Количество треугольников в графе
6. Краткие выводы о данных.
7. БОНУСЫ - сделайте красивую визулизацию или расчет дополнительных метрик с выводами и вы получите дополнительные баллы за задание

In [2]:
edges = gl.SFrame.read_csv('the_graph.csv', delimiter=',', verbose=False)
g = gl.SGraph().add_edges(edges, src_field='src', dst_field='dst')

This non-commercial license of GraphLab Create for academic use is assigned to matushkin@phystech.edu and will expire on November 06, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\ASUS-PC\AppData\Local\Temp\graphlab_server_1479234964.log.0


In [3]:
def print_graph_info():
    vert_edgs = g.summary()
    print 'Number of vertices: {0}'.format(vert_edgs['num_vertices'])
    print 'Number of edges: {0}'.format(vert_edgs['num_edges'])
    
    deg = sorted(gl.degree_counting.create(g).graph.get_vertices()['total_degree'], reverse=True)
    plt.hist(deg, bins=20, log=True, facecolor='red', alpha=0.75)
    plt.xlabel('Degree')
    plt.ylabel('Number of vertices')
    plt.title('Histogram of degree distribution')
    # plt.xscale('log')
    plt.show()
    
    print 'Graph density: %f' % (float(vert_edgs['num_edges']) / vert_edgs['num_vertices'])
    print
    
    G = nx.Graph()
    G.add_nodes_from(range(vert_edgs['num_vertices']))
    G.add_edges_from(g.get_edges().to_numpy())
    eccs = nx.eccentricity(G, v=np.random.choice(range(vert_edgs['num_vertices']), size=100, replace=False))
    print 'Diameter of graph >= ', max(eccs.itervalues())
    # print 'Diameter of graph: %d' % nx.diameter(G)
    
    print 'Number of triangles: %d' % gl.triangle_counting.create(g, verbose=False).num_triangles

#### Найдем ряд базовых статистик

In [ ]:
print_graph_info()

Выводы: большой разреженный граф с малым диаметром.

----
### 2. Генерация дополнительных данных - 20 Баллов

Перед нами стоит задача создания модели классификации, которая в дальнейшем будет использоваться для повышения разнообразия блока рекомендаций. Для большинства моделей классификации требуется минимум 2 класса объектов - негативный и позитивный. Но у нас есть только граф, ребра которые в нем присутствуют это позитивные примеры. Получается, что у нас нет негативных примеров. 

Ответьте на 3 вопроса:

1. Можем ли мы сами создать негативные примеры? Граф это описание связей между вершинами, если мы будем случайным образом выбирать две вершины и считать, что это ребро - негативный пример, имеет ли это смысл?
2. Если мы решим сгенерировать негативные примеры, как должна быть устроена процедура генерации, чтобы обобщаяющая способность модели была наилучшей?
3. Как зависит обобщающая способность модели от негативных примеров, которые мы ей покажем?

1. Можем. Имеет, но это не лучший способ.
2. Если каждый раз брать две случайные вершины, то мы часто будем получать вершины из разных кластеров, про которые "и так понятно", что они не соединены ребром. Стоит взять побольше негативных примеров, которые похожи на позитивные.
3. Отрицательные примеры, которые мы выделим на train-графе должны быть похожи на большинство пар несмежных вершин, тогда, возможно, все будет ОК, иначе модель будет обладать низкой обучающей способностью.

Создадим класс отрицательных примеров, сгенерировав его самым простым образом: берем две вершины, если ребро между ними отсутствует в графе, то это отрицательный пример.

1. Попробуйте улучшить генерацию отрицательных примеров

In [5]:
vert_edgs = g.summary()
vert = gl.degree_counting.create(g).graph.get_vertices()

# read in existing edges
graph_file = open('the_graph.csv')
graph_file.readline()
existing_edges = set()

for x in graph_file:
    start, end = x.split(',')
    start, end = int(start), int(end)
    existing_edges.add((start, end))

In [6]:
# generating negative examples, so that class balance is 50/50 
generated_nonexisting_edges = []
counter = 0
while True:
    start = np.random.randint(0, vert_edgs['num_vertices'])
    end = np.random.randint(0, vert_edgs['num_vertices'])
    if (start != end) & ((start, end) not in existing_edges):
        generated_nonexisting_edges.append([start, end, 0])
        
    counter += 1
    if counter == vert_edgs['num_edges']:
        break

# create SFrame with negative examples
generated_nonexisting_edges = pd.DataFrame(data=generated_nonexisting_edges, columns=['src', 'dst', 'class'])
generated_nonexisting_edges = gl.SFrame(data=generated_nonexisting_edges)

# add target function to the original dataset 
edges['class'] = [1] * edges.shape[0]

# add negative examples to the main data and shuffle
edges = edges.append(generated_nonexisting_edges)
edges = gl.cross_validation.shuffle(edges)

# update our graph g with fake edges
g = gl.SGraph().add_edges(edges, src_field='src', dst_field='dst')

vert_edgs = g.summary()

print 'Number of vertices = {0}'.format(vert_edgs['num_vertices'])
print 'Number of edges = {0}'.format(vert_edgs['num_edges'])

Number of vertices = 262111
Number of edges = 2769704


----
### 3. Создание матрицы признаков для ребер - 30 Баллов

Для создания модели классификации нам необходимы признаки, которые описывают каждое ребро. Вот базовый список того, что можно посчитать:

Для вершин:

1. Список и количество вершин, из которых ребра приходят в данную вершину
2. Список и количество вершин, в которые ребра приходят из данной вершины
3. Список и количество вершин связанных с данной вершиной
4. Список и количество вершин, которые связанны с данной вершиной как входящими, так и исходящими ребрами

Также можно добавить: вершины с которыми данная вершина образует треугольники, кластеризовать вершины и использовать кластер данной вершины и др.

Придумайте дополнительные интересные признаки для вершин и вы получите дополнительный балл за домашнюю работу!

In [7]:
all_vertices = g.get_vertices()
_ = all_vertices.rename({"__id": "id"})

# calculating each vertices in and out connections
out_vertices = edges.groupby("src", {"out_vertices": gl.aggregate.CONCAT("dst")})
out_vertices.rename({"src": "id"})

in_vertices = edges.groupby("dst", {"in_vertices": gl.aggregate.CONCAT("src")})
in_vertices.rename({"dst": "id"})
print

Мы получили базовый набор данных - список входящих и исходящих вершин, для каждой вершины. Теперь найдем количества вершин для метрик 1 и 2. Метрики 3 и 4 вы посчитаете сами

Рассчитаем метрики 3 и 4, вычислим коэффициенты Жаккара и Адамик-Адара, а также косинусную меру для списков из метрик 1-4.

In [8]:
all_vertices = all_vertices.join(in_vertices, on="id", how="outer")
all_vertices = all_vertices.fillna('in_vertices', [])
all_vertices['in_degree'] = all_vertices["in_vertices"].apply(lambda x: len(x))

all_vertices = all_vertices.join(out_vertices, on="id", how="outer")
all_vertices = all_vertices.fillna('out_vertices', [])
all_vertices['out_degree'] = all_vertices["out_vertices"].apply(lambda x: len(x))

all_vertices['adj_vertices'] =  all_vertices.apply(lambda x : list(set(x['in_vertices']) | set(x['out_vertices'])))
all_vertices['degree'] =  all_vertices['adj_vertices'].apply(lambda x: len(x))

all_vertices['dadj_vertices'] =  all_vertices.apply(lambda x : list(set(x['in_vertices']) & set(x['out_vertices'])))
all_vertices['ddegree'] = all_vertices['dadj_vertices'].apply(lambda x: len(x))

In [9]:
pr = gl.pagerank.create(g, verbose=False)
pr_out = pr['pagerank']
_ = pr_out.rename({'__id' : 'id'})

tc = gl.triangle_counting.create(g, verbose=False)
tc_out = tc['triangle_count']
_ = tc_out.rename({'__id' : 'id'})

In [10]:
all_vertices = all_vertices.join(tc_out, on='id', how='left')
all_vertices = all_vertices.join(pr_out, on='id', how='left')
all_vertices.remove_column('delta')

id,in_vertices,in_degree,out_vertices,out_degree
251434,"[186785, 57105, 187707,255119, 250137, 173240, ...",11,"[109983, 213798, 240147,120729, 77913, 255119, ...",12
211023,"[69431, 223453, 8927,15744, 130553, 111522, ...",11,"[224381, 42617, 209929,190100, 34545, 147802, ...",17
21855,"[217557, 49405, 191808,25116, 39961, 38455, ...",13,"[208698, 25114, 21854,25116, 15075, 25115, ...",7
233270,"[105601, 93151, 29698,229329, 127229, 254122] ...",6,"[138297, 139583, 12446,258684, 114321, 10760, ...",12
88004,"[88002, 186039, 130339,88003, 132168, 72281, ...",9,"[183896, 88003, 88002,204086, 64573, 72281, ...",8
79732,"[103248, 183618, 198593,105532, 165700, 81408, ...",19,"[79734, 75040, 81407,79730, 83186, 231402] ...",6
63664,"[116067, 63666, 58475,80726, 195309, 187155, ...",10,"[63665, 110648, 46916,22409, 189676, 80726, ...",8
127950,"[127952, 251251, 163381,175281, 8107, 98108, ...",8,"[127952, 110645, 98108,48378, 17388, 93037, ...",14
7899,"[167871, 15258, 7898,65076, 6150, 8977, 7896, ...",13,"[160221, 13587, 104872,229593, 17454, 8977, ...",13
25263,"[111709, 139715, 42349,49187, 36274, 174885, ...",11,"[42349, 201015, 42350,241384, 6965, 6961, 6 ...",9


In [11]:
all_vertices.head(1)

Columns:
	id	int
	in_vertices	list
	in_degree	int
	out_vertices	list
	out_degree	int
	adj_vertices	array
	degree	int
	dadj_vertices	array
	ddegree	int
	triangle_count	int
	pagerank	float

Rows: 1

Data:
+--------+-------------------------------+-----------+
|   id   |          in_vertices          | in_degree |
+--------+-------------------------------+-----------+
| 251434 | [186785, 57105, 187707, 25... |     11    |
+--------+-------------------------------+-----------+
+-------------------------------+------------+-------------------------------+
|          out_vertices         | out_degree |          adj_vertices         |
+-------------------------------+------------+-------------------------------+
| [109983, 213798, 240147, 1... |     12     | [126019.0, 91141.0, 255120... |
+-------------------------------+------------+-------------------------------+
+--------+----------------------+---------+----------------+---------------+
| degree |    dadj_vertices     | ddegree | triangle_count |    pagerank   |
+--------+----------------------+---------+----------------+---------------+
|   21   | [250137.0, 255119.0] |    2    |       0        | 1.05661299071 |
+--------+----------------------+---------+----------------+---------------+
[1 rows x 11 columns]

In [12]:
edges.head(1)

src,dst,class
12062,161043,0


Мы получили характеристики для каждой вершины графа. Так как у нас задача предсказания наличия ребра, мы должны составить описание каждого ребра, используя метрики вершин. После составления характеристик ребер, мы можем перейти к генерации свойств ребер, взяв за основу метрики вершин ребра. Пример метрик для ребер, которые мы можем найти:

1. Общие друзья вершин ребра
2. Общее количество друзей вершин ребра
3. Коэффициенты Жаккара, Пирсона, Адамик - Адара, косинусная мера

Данные метрики находятся для каждого списка вершин, пунктов 1 - 4

In [13]:
edges = edges.join(pr_out, on={"src": "id"}, how="right")

In [14]:
edges = edges.join(all_vertices, on={"src": "id"}, how="right")
_ = edges.rename({"in_vertices": "src_in_vertices", "out_vertices": "src_out_vertices",
              "in_degree": "src_in_degree", "out_degree": "src_out_degree",
              "adj_vertices": "src_adj_vertices", "dadj_vertices": "src_dadj_vertices",
              "degree": "src_degree", "ddegree": "src_ddegree",
              "triangle_count" : "src_triangle_count", "pagerank" : "src_pagerank"})

edges = edges.join(all_vertices, on={"dst": "id"}, how="right")
_ = edges.rename({"in_vertices": "dst_in_vertices", "out_vertices": "dst_out_vertices",
              "in_degree": "dst_in_degree", "out_degree": "dst_out_degree",
              "adj_vertices": "dst_adj_vertices", "dadj_vertices": "dst_dadj_vertices",
              "degree": "dst_degree", "ddegree": "dst_ddegree",
              "triangle_count" : "dst_triangle_count", "pagerank" : "dst_pagerank"})

Найдем пункт 1 - общих друзей для списков вершин из пункта 1. Пункты 2 и 3 вы посчитаете самостоятельно

In [15]:
def common_friends(u, v, u_neighbors, v_neighbors):
    u_neighbors = set(u_neighbors)
    if v in u_neighbors:
        u_neighbors.remove(v)

    v_neighbors = set(v_neighbors)
    if u in v_neighbors:
        v_neighbors.remove(u)
        
    return len(u_neighbors & v_neighbors)

In [16]:
edges['common_in_vertices'] = (edges[['src', 'dst', 'src_in_vertices', 'dst_in_vertices']]
                               .apply(lambda x: common_friends(x['src'],
                                                               x['dst'],
                                                               x['src_in_vertices'],
                                                               x['dst_in_vertices'])))

edges['common_out_vertices'] = (edges[['src', 'dst', 'src_out_vertices', 'dst_out_vertices']]
                                .apply(lambda x: common_friends(x['src'],
                                                                x['dst'],
                                                                x['src_out_vertices'],
                                                                x['dst_out_vertices'])))

edges['common_adj_vertices'] = (edges[['src', 'dst', 'src_adj_vertices', 'dst_adj_vertices']]
                                .apply(lambda x: common_friends(x['src'],
                                                                x['dst'],
                                                                x['src_adj_vertices'],
                                                                x['dst_adj_vertices'])))

edges['common_dadj_vertices'] = (edges[['src', 'dst', 'src_dadj_vertices', 'dst_dadj_vertices']]
                                 .apply(lambda x: common_friends(x['src'],
                                                                 x['dst'],
                                                                 x['src_dadj_vertices'],
                                                                 x['dst_dadj_vertices'])))

In [17]:
def Jaccard(u, v, u_neighbors, v_neighbors):
    u_neighbors = set(u_neighbors)
    if v in u_neighbors:
        u_neighbors.remove(v)
    v_neighbors = set(v_neighbors)
    if u in v_neighbors:
        v_neighbors.remove(u)
    if len(u_neighbors) + len(v_neighbors) == 0:
        return 0
    else:
        return len(u_neighbors & v_neighbors) / float(len(u_neighbors | v_neighbors))

In [18]:
edges['Jaccard_in_vertices'] = (edges[['src', 'dst', 'src_in_vertices', 'dst_in_vertices']]
                                .apply(lambda x: Jaccard(x['src'],
                                                         x['dst'],
                                                         x['src_in_vertices'],
                                                         x['dst_in_vertices'])))

edges['Jaccard_out_vertices'] = (edges[['src', 'dst', 'src_out_vertices', 'dst_out_vertices']]
                                 .apply(lambda x: Jaccard(x['src'],
                                                          x['dst'],
                                                          x['src_out_vertices'],
                                                          x['dst_out_vertices'])))

edges['Jaccard_adj_vertices'] = (edges[['src', 'dst', 'src_adj_vertices', 'dst_adj_vertices']]
                                 .apply(lambda x: Jaccard(x['src'],
                                                          x['dst'],
                                                          x['src_adj_vertices'],
                                                          x['dst_adj_vertices'])))

edges['Jaccard_dadj_vertices'] = (edges[['src', 'dst', 'src_dadj_vertices', 'dst_dadj_vertices']]
                                  .apply(lambda x: Jaccard(x['src'],
                                                           x['dst'],
                                                           x['src_dadj_vertices'],
                                                           x['dst_dadj_vertices'])))

In [19]:
def cosine(u, v, u_neighbors, v_neighbors):
    u_neighbors = set(u_neighbors)
    if v in u_neighbors:
        u_neighbors.remove(v)
    v_neighbors = set(v_neighbors)
    if u in v_neighbors:
        v_neighbors.remove(u)
    inter_len = len(u_neighbors & v_neighbors)
    if len(u_neighbors) + len(v_neighbors) == 0:
        return 0
    else:
        return inter_len / np.sqrt(len(u_neighbors) + len(v_neighbors))

In [20]:
edges['cosine_in_vertices'] = (edges[['src', 'dst', 'src_in_vertices', 'dst_in_vertices']]
                               .apply(lambda x: cosine(x['src'],
                                                       x['dst'],
                                                       x['src_in_vertices'],
                                                       x['dst_in_vertices'])))

edges['cosine_out_vertices'] = (edges[['src', 'dst', 'src_out_vertices', 'dst_out_vertices']]
                                .apply(lambda x: cosine(x['src'],
                                                        x['dst'],
                                                        x['src_out_vertices'],
                                                        x['dst_out_vertices'])))

In [21]:
vert = gl.degree_counting.create(g).graph.get_vertices()
def aa_coef(u, v, u_neighbors, v_neighbors):
    u_neighbors = set(u_neighbors)
    if v in u_neighbors:
        u_neighbors.remove(v)
    v_neighbors = set(v_neighbors)
    if u in v_neighbors:
        v_neighbors.remove(u)
    inter = u_neighbors & v_neighbors
    union = u_neighbors | v_neighbors
    return sum([1 / np.log(vert[x]['total_degree'] + 1) for x in inter])

In [22]:
edges['aa_coef_in_vertices'] = (edges[['src', 'dst', 'src_in_vertices', 'dst_in_vertices']]
                                .apply(lambda x: aa_coef(x['src'],
                                                         x['dst'],
                                                         x['src_in_vertices'],
                                                         x['dst_in_vertices'])))

edges['aa_coef_out_vertices'] = (edges[['src', 'dst', 'src_out_vertices', 'dst_out_vertices']]
                                .apply(lambda x: aa_coef(x['src'],
                                                         x['dst'],
                                                         x['src_out_vertices'],
                                                         x['dst_out_vertices'])))

In [23]:
edges_copy = edges

In [25]:
_ = edges.remove_column('Jaccard_adj_vertices')
_ = edges.remove_column('Jaccard_dadj_vertices')

src,dst,class,src_pagerank,delta,src_in_vertices,src_in_degree
12062,161043,0,1.17889216687,3.12804113078e-06,"[65240, 12061, 173956,14868, 5122, 44596, ...",13
180927,56278,0,0.690763410829,1.59269827582e-06,"[112081, 156549, 192368,71897, 154667, 162750, ...",7
104396,158911,0,0.799518756983,1.98933159801e-06,"[171799, 146162, 63259,198419, 35719, 9490, ...",8
42025,45522,1,1.47833525909,4.02305239167e-06,"[176359, 42026, 45523,29851, 45522, 156807, ...",18
45789,255791,0,1.46845972674,4.12709697728e-06,"[183407, 52393, 181594,33796, 45791, 119038, ...",14
167861,11160,0,0.634121788926,1.59558948609e-06,"[47167, 109781, 26729,47716, 111536] ...",5
172452,169229,0,0.880193952516,2.12870329075e-06,"[225573, 118191, 210934,140136, 74899, 114137, ...",12
79746,128920,0,1.61258672216,4.49333374597e-06,"[97557, 9443, 78143,233036, 252030, 78141, ...",18
24689,128125,0,1.78389190402,4.89254858049e-06,"[100231, 29305, 18477,116225, 53753, 67816, ...",18
13173,63238,0,0.755399336374,1.76922655082e-06,"[10116, 54996, 13002,193913, 23262, 34786, ...",8


In [26]:
_ = edges.remove_column('src_ddegree')
_ = edges.remove_column('dst_ddegree')
_ = edges.remove_column('src_degree')
_ = edges.remove_column('dst_degree')
_ = edges.remove_column('common_dadj_vertices')
_ = edges.remove_column('common_adj_vertices')

In [27]:
_ = edges.remove_column('src_in_vertices')
_ = edges.remove_column('src_out_vertices')
_ = edges.remove_column('src_adj_vertices')
_ = edges.remove_column('src_dadj_vertices')
_ = edges.remove_column('dst_in_vertices')
_ = edges.remove_column('dst_out_vertices')
_ = edges.remove_column('dst_adj_vertices')
_ = edges.remove_column('dst_dadj_vertices')

In [28]:
_ = edges.remove_column('src_in_degree')
_ = edges.remove_column('src_out_degree')
_ = edges.remove_column('dst_in_degree')
_ = edges.remove_column('dst_out_degree')
_ = edges.remove_column('cosine_in_vertices')
_ = edges.remove_column('cosine_out_vertices')

In [29]:
_ = edges.remove_column('Jaccard_in_vertices')
_ = edges.remove_column('Jaccard_out_vertices')

In [31]:
_ = edges.remove_column('delta')

Результатом пункта 3 является матрица признаков. Теперь мы готовы занятся настройкой моделей. Главная ваша задача в пункте 3 - подготовить как можно больше качественных признаков, которые могли бы использоваться для настройки модели машинного обучения.

In [32]:
edges.head(1)

Columns:
	src	int
	dst	int
	class	int
	src_pagerank	float
	src_triangle_count	int
	pagerank.1	float
	dst_triangle_count	int
	dst_pagerank	float
	common_in_vertices	int
	common_out_vertices	int
	aa_coef_in_vertices	float
	aa_coef_out_vertices	float

Rows: 1

Data:
+-------+--------+-------+---------------+--------------------+---------------+
|  src  |  dst   | class |  src_pagerank | src_triangle_count |   pagerank.1  |
+-------+--------+-------+---------------+--------------------+---------------+
| 12062 | 161043 |   0   | 1.17889216687 |         3          | 1.17889216687 |
+-------+--------+-------+---------------+--------------------+---------------+
+--------------------+---------------+--------------------+---------------------+
| dst_triangle_count |  dst_pagerank | common_in_vertices | common_out_vertices |
+--------------------+---------------+--------------------+---------------------+
|         4          | 1.11630945097 |         0          |          0          |
+--------------------+---------------+--------------------+---------------------+
+---------------------+----------------------+
| aa_coef_in_vertices | aa_coef_out_vertices |
+---------------------+----------------------+
|         0.0         |         0.0          |
+---------------------+----------------------+
[1 rows x 12 columns]

----
### 4. Настройка модели и валидация качества модели - 20 Баллов

Исключим подозрительные ребра из рассмотрения, а затем разобьем датасет на 2 части, для обучения и проверки результатов. Настроим базовую модель классификации - логистическую регрессию.
Для улучшения качества модели вам предстоит выполнить следующие пункты:

1. Кросс - валидация для настройки гиперпараметров модели и регуляризации
2. Подбор модели машинного обучения (случайный лес, бустинг, нейронная сеть и т.д.)

In [33]:
susp = gl.SFrame.read_csv('suspicions.csv', delimiter=',', verbose=False)

In [ ]:
#edges = gl.SFrame.read_csv('edges.csv', delimiter=',', verbose=False)

In [ ]:
#pdframe = edges.to_dataframe()

In [ ]:
#pdframe.to_csv("unmerged.csv")

In [ ]:
#edges.export_csv("unmerged2.csv")

In [ ]:
#edges = gl.SFrame.read_csv('edges.csv', delimiter=',', verbose=False)

In [ ]:
print edges.shape
edges = edges.join(susp, on=['src', 'dst'], how='left')
print edges.shape

(2769704, 12)


In [ ]:
#pdframe = edges.to_dataframe()
#pdframe.to_csv("merged.csv")

In [ ]:
clean_edges = edges[edges['edge_id'] == None]
susp_edges = edges[edges['edge_id'] != None]
print clean_edges.shape, susp_edges.shape

In [ ]:
clean_edges.remove_column('edge_id')
clean_edges.remove_column('src')
clean_edges.remove_column('dst')
print

In [ ]:
clean_edges.head(1)

In [ ]:
# split on train and test
test, train = clean_edges.random_split(0.2)

In [ ]:
print test.shape, train.shape

In [ ]:
#pdframe = clean_edges.to_dataframe()
#pdframe.to_csv("data2.csv")

In [ ]:
#pdframe.to_csv("data2.csv")

In [ ]:
#clean_edges = gl.SFrame.read_csv('data.csv', delimiter=',', verbose=False)

In [ ]:
model = gl.classifier.logistic_classifier.create(clean_edges, 
                                                 target="class", max_iterations=10, 
                                                 features=[x for x in clean_edges.column_names() if 'class' not in x])

In [ ]:
model.coefficients.print_rows(num_rows=21)

In [ ]:
model = gl.classifier.boosted_trees_classifier.create(clean_edges, 
                                                 target="class", max_iterations=100,
                                                 features=[x for x in clean_edges.column_names() if 'class' not in x])

In [ ]:
# fit basic classification model - LR
model = gl.classifier.boosted_trees_classifier.create(train, 
                                                 target='class',
                                                 features=[x for x in clean_edges.column_names() if 'class' not in x])
results = model.evaluate(test)
print results

----
### 5. Отправка результатов в контест на Kaggle - 20 Баллов


Завершая домашнюю работу, нам небходимо предсказать вероятности наличия ребер, для заданного списка ребер. Результат отправлется в контест на kaggle.com в формате: edge_id - probability

In [ ]:
predicted = model.predict(susp_edges, output_type='probability')

In [ ]:
print predicted[0:10]

In [ ]:
pred = susp_edges['class']

In [ ]:
susp_edges['probability'] = predicted

In [ ]:
result = susp_edges['edge_id', 'probability']

In [ ]:
result.save('result8.csv', format='csv')